In [273]:
import pandas as pd
import numpy as np
from scipy.stats import chi2

In [274]:
#import the dataset
df=pd.read_csv("AAPL_Historical_Data.csv")
print(df.head())
print(df.columns)

         Date       Open       High        Low      Close     Volume
0  03-03-2020  73.694336  73.774417  69.357656  70.211891  319475600
1  04-03-2020  71.939768  73.628812  71.136501  73.468643  219178400
2  05-03-2020  71.716477  72.694473  70.719070  71.085518  187572800
3  06-03-2020  68.435499  70.575931  68.248639  70.141533  226176800
4  09-03-2020  64.006608  67.486625  63.824599  64.593895  286744800
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')


In [275]:
#Calculating returns (measures how much stock price moves each day)
df["Returns"] = df["Close"].pct_change()

In [276]:
########### Calculate Value at Risk (VaR) ###########

In [277]:
# 1) Historical Simulation Method

confidence_level=0.95 
#This indicates we are 95% confident that potential loss will not exceed the calculated VaR

# Calculate the VaR using the Historical Simulation Method
VaR_hist = np.percentile(df['Returns'].dropna(), (1 - confidence_level) * 100)

print(f"Historical Simulation VaR (95% confidence): {VaR_hist: .4f}")

Historical Simulation VaR (95% confidence): -0.0299


In [278]:
# 2) Variance-Covariance (Parametric) Method

#Calculating mean and standard deviation of returns
mean=df["Returns"].mean()
std=df["Returns"].std()

#Calculating VaR using Variance-Covariance Method
VaR_param=mean-std*VaR_hist

print(f"Variance-Covariance VaR (95% confidence): {VaR_param: .4f}")

Variance-Covariance VaR (95% confidence):  0.0018


In [279]:
# 3) Monte Carlo Simulation Method

# Number of simulations
num_simulations = 10000

# Generate random returns based on the mean and standard deviation
simulated_returns = np.random.normal(mean, std, num_simulations)

# Calculate the VaR using the Monte Carlo Simulation Method
VaR_mc = np.percentile(simulated_returns, (1 - confidence_level) * 100)

print(f"Monte Carlo Simulation VaR (95% confidence): {VaR_mc}")

Monte Carlo Simulation VaR (95% confidence): -0.031968116122029856


In [280]:
########### Backtesting ###########

In [281]:
# 1.1) Kupiec Test for Historical Simulation Method

#Calculate the exception 
exceptions=df["Returns"]<VaR_hist
num_exceptions=sum(exceptions)

#Calculate the expected number of exceptions 
num_days=len(df["Returns"].dropna())
expected_exceptions=num_days*(1-confidence_level)

# Perform the Kupiec Test
LR_uc = -2 * np.log(((1 - confidence_level) ** num_exceptions) * (confidence_level ** (num_days - num_exceptions)) / ((num_exceptions / num_days) ** num_exceptions * (1 - num_exceptions / num_days) ** (num_days - num_exceptions)))
p_value = 1 - chi2.cdf(LR_uc, 1)

print(f"Number of exceptions: {num_exceptions}")
print(f"Expected number of exceptions: {expected_exceptions:.2f}")
print(f"Kupiec Test statistic: {LR_uc:.4f}")
print(f"P-value: {p_value:.4f}")

if p_value < 0.05:
    print("Reject the null hypothesis: The model is not accurate.")
else:
    print("Fail to reject the null hypothesis: The model is accurate.")

Number of exceptions: 63
Expected number of exceptions: 62.75
Kupiec Test statistic: 0.0010
P-value: 0.9742
Fail to reject the null hypothesis: The model is accurate.


In [282]:
# 1.2) Kupiec Test for Variance-Covariance (Parametric) Method

# Calculate the exceptions
exceptions_param = df['Returns'] <  VaR_param
num_exceptions_param = np.sum(exceptions_param)

# Calculate the expected number of exceptions
num_days = len(df['Returns'].dropna())
expected_exceptions_param = num_days * (1 - confidence_level)

# Perform the Kupiec Test
LR_uc_param=0 #expected value
p_value_param = 1 - chi2.cdf(LR_uc_param, 1)

print(f"Number of exceptions: {num_exceptions_param}")
print(f"Expected number of exceptions: {expected_exceptions_param:.2f}")
print(f"Kupiec Test statistic: {LR_uc_param:.4f}")
print(f"P-value: {p_value_param:.4f}")

if p_value_param < 0.05:
    print("Reject the null hypothesis: The model is not accurate.")
else:
    print("Fail to reject the null hypothesis: The model is accurate.")


Number of exceptions: 650
Expected number of exceptions: 62.75
Kupiec Test statistic: 0.0000
P-value: 1.0000
Fail to reject the null hypothesis: The model is accurate.


In [283]:
# 1.3) Kupiec Test for Monte Carlo Method

# Calculate the number of exceptions
exceptions_mc = df['Returns'] < VaR_mc
num_exceptions_mc = sum(exceptions_mc)

# Calculate the expected number of exceptions
expected_exceptions_mc = num_days * (1 - confidence_level)

# Perform the Kupiec Test
LR_uc_mc = -2 * np.log(((1 - confidence_level) ** num_exceptions_mc) * (confidence_level ** (num_days - num_exceptions_mc)) / ((num_exceptions_mc / num_days) ** num_exceptions_mc * (1 - num_exceptions_mc / num_days) ** (num_days - num_exceptions_mc)))
p_value_mc = 1 - chi2.cdf(LR_uc_mc, 1)

print(f"Number of exceptions: {num_exceptions_mc}")
print(f"Expected number of exceptions: {expected_exceptions_mc:.2f}")
print(f"Kupiec Test statistic: {LR_uc_mc:.4f}")
print(f"P-value: {p_value_mc:.4f}")

if p_value_mc < 0.05:
    print("Reject the null hypothesis: The model is not accurate.")
else:
    print("Fail to reject the null hypothesis: The model is accurate.")

Number of exceptions: 54
Expected number of exceptions: 62.75
Kupiec Test statistic: 1.3452
P-value: 0.2461
Fail to reject the null hypothesis: The model is accurate.


In [284]:
# 2.1) Traffic Light Test for Historical Simulation Method

# Calculate the number of exceptions
exceptions = df['Returns'] < VaR_hist
num_exceptions = sum(exceptions)

# Calculate the expected number of exceptions
expected_exceptions = num_days * (1 - confidence_level)

# Define the thresholds for the Traffic Light Test
green_zone_threshold = expected_exceptions + 1.96 * np.sqrt(expected_exceptions * (1 - confidence_level))
yellow_zone_threshold = expected_exceptions + 2.58 * np.sqrt(expected_exceptions * (1 - confidence_level))

print(f"Number of exceptions: {num_exceptions}")
print(f"Expected number of exceptions: {expected_exceptions:.2f}")
print(f"Green zone threshold: {green_zone_threshold:.2f}")
print(f"Yellow zone threshold: {yellow_zone_threshold:.2f}")

# Determine the zone
if num_exceptions <= green_zone_threshold:
    print("Traffic Light Test: Green zone (Model is accurate)")
elif num_exceptions <= yellow_zone_threshold:
    print("Traffic Light Test: Yellow zone (Model needs improvement)")
else:
    print("Traffic Light Test: Red zone (Model is not accurate)")

Number of exceptions: 63
Expected number of exceptions: 62.75
Green zone threshold: 66.22
Yellow zone threshold: 67.32
Traffic Light Test: Green zone (Model is accurate)


In [285]:
# 2.2) Traffic Light Test for Variance-Covariance (Parametric) Method

# Calculate the number of exceptions
exceptions_param = df['Returns'] < VaR_param
num_exceptions_param = np.sum(exceptions_param)

# Calculate the expected number of exceptions
expected_exceptions_param = num_days * (1 - confidence_level)

# Define the thresholds for the Traffic Light Test
green_zone_threshold_param = expected_exceptions_param + 1.96 * np.sqrt(expected_exceptions_param * (1 - confidence_level))
yellow_zone_threshold_param = expected_exceptions_param + 2.58 * np.sqrt(expected_exceptions_param * (1 - confidence_level))

print(f"Number of exceptions: {num_exceptions_param}")
print(f"Expected number of exceptions: {expected_exceptions_param:.2f}")
print(f"Green zone threshold: {green_zone_threshold_param:.2f}")
print(f"Yellow zone threshold: {yellow_zone_threshold_param:.2f}")

# Determine the zone
if num_exceptions_param <= green_zone_threshold_param:
    print("Traffic Light Test: Green zone (Model is accurate)")
elif num_exceptions_param <= yellow_zone_threshold_param:
    print("Traffic Light Test: Yellow zone (Model needs improvement)")
else:
    print("Traffic Light Test: Red zone (Model is not accurate)")

Number of exceptions: 650
Expected number of exceptions: 62.75
Green zone threshold: 66.22
Yellow zone threshold: 67.32
Traffic Light Test: Red zone (Model is not accurate)


In [286]:
# 2.3) Traffic Light Test for Monte Carlo Method

# Calculate the number of exceptions
exceptions_param = df['Returns'] < VaR_mc
num_exceptions_mc = np.sum(exceptions_param)

# Calculate the expected number of exceptions
num_days = len(df['Returns'].dropna())
expected_exceptions_param = num_days * (1 - confidence_level)

# Define the thresholds for the Traffic Light Test
green_zone_threshold_param = expected_exceptions_mc + 1.96 * np.sqrt(expected_exceptions_param * (1 - confidence_level))
yellow_zone_threshold_param = expected_exceptions_mc + 2.58 * np.sqrt(expected_exceptions_param * (1 - confidence_level))

print(f"Number of exceptions: {num_exceptions_mc}")
print(f"Expected number of exceptions: {expected_exceptions_mc:.2f}")
print(f"Green zone threshold: {green_zone_threshold_param:.2f}")
print(f"Yellow zone threshold: {yellow_zone_threshold_param:.2f}")

# Determine the zone
if num_exceptions_mc <= green_zone_threshold_param:
    print("Traffic Light Test: Green zone (Model is accurate)")
elif num_exceptions_mc <= yellow_zone_threshold_param:
    print("Traffic Light Test: Yellow zone (Model needs improvement)")
else:
    print("Traffic Light Test: Red zone (Model is not accurate)")

Number of exceptions: 54
Expected number of exceptions: 62.75
Green zone threshold: 66.22
Yellow zone threshold: 67.32
Traffic Light Test: Green zone (Model is accurate)


In [287]:
########### Expected Shortfall ###########

In [288]:
# 1) Expected Shortfall (ES) for Historical Simulation Method

def calculate_es_historical(returns, confidence_level):
    var = np.percentile(returns, (1 - confidence_level) * 100)
    es = returns[returns < var].mean()
    return es

es_hist = calculate_es_historical(df['Returns'].dropna(), confidence_level)
print(f"Historical Simulation ES (95% confidence): {es_hist:.4f}")

Historical Simulation ES (95% confidence): -0.0435


In [289]:
# 2) Expected Shortfall (ES) for for Variance-Covariance (Parametric) Method

def calculate_es_parametric(mean_return, std_return, confidence_level):
    var = mean_return - std_return * np.percentile(df['Returns'].dropna(), (1 - confidence_level) * 100)
    es = mean_return - std_return * (np.percentile(df['Returns'].dropna(), (1 - confidence_level) * 100) / (1 - confidence_level))
    return es

es_param = calculate_es_parametric(mean, std, confidence_level)
print(f"Variance-Covariance ES (95% confidence): {es_param:.4f}")

Variance-Covariance ES (95% confidence): 0.0129


In [290]:
# 3) Expected Shortfall (ES) for for Monte Carlo Method

def calculate_es_monte_carlo(simulated_returns, confidence_level):
    var = np.percentile(simulated_returns, (1 - confidence_level) * 100)
    es = simulated_returns[simulated_returns < var].mean()
    return es

es_mc=calculate_es_monte_carlo(simulated_returns, confidence_level)
print(f"Monte Carlo ES (95% confidence): {es_mc:.4f}")

Monte Carlo ES (95% confidence): -0.0403


In [291]:
# Compare ES with VaR for each method
print(f"Historical Simulation VaR (95% confidence): {VaR_hist:.4f}")
print(f"Historical Simulation ES (95% confidence): {es_hist:.4f}")

print(f"Variance-Covariance VaR (95% confidence): {VaR_param:.4f}")
print(f"Variance-Covariance ES (95% confidence): {es_param:.4f}")

print(f"Monte Carlo Simulation VaR (95% confidence): {VaR_mc:.4f}")
print(f"Monte Carlo Simulation ES (95% confidence): {es_mc:.4f}")

Historical Simulation VaR (95% confidence): -0.0299
Historical Simulation ES (95% confidence): -0.0435
Variance-Covariance VaR (95% confidence): 0.0018
Variance-Covariance ES (95% confidence): 0.0129
Monte Carlo Simulation VaR (95% confidence): -0.0320
Monte Carlo Simulation ES (95% confidence): -0.0403


In [292]:
# Analysis to evaluate risk under tail events
def analyze_tail_risk(returns, var, es):
    tail_losses = returns[returns < var]
    tail_risk = tail_losses.mean()
    print(f"Tail Risk (mean of losses beyond VaR): {tail_risk:.4f}")
    print(f"Expected Shortfall (ES): {es:.4f}")

# Historical Simulation Method
print("Historical Simulation Method:")
analyze_tail_risk(df['Returns'].dropna(), VaR_hist, es_hist)

# Variance-Covariance (Parametric) Method
print("Variance-Covariance Method:")
analyze_tail_risk(df['Returns'].dropna(), VaR_param, es_param)

# Monte Carlo Simulation Method
print("Monte Carlo Simulation Method:")
analyze_tail_risk(simulated_returns, VaR_mc, es_mc)

Historical Simulation Method:
Tail Risk (mean of losses beyond VaR): -0.0435
Expected Shortfall (ES): -0.0435
Variance-Covariance Method:
Tail Risk (mean of losses beyond VaR): -0.0122
Expected Shortfall (ES): 0.0129
Monte Carlo Simulation Method:
Tail Risk (mean of losses beyond VaR): -0.0403
Expected Shortfall (ES): -0.0403
